In [1]:
import os 
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import warnings
warnings.simplefilter('ignore')

In [2]:
print(os.getcwd())


/Users/ellenwu/homework2-2/submission3/data-code


In [ ]:
print(os.path.exists('/Users/ellenwu/homework2-2/data/*/input/HCRISv1996'))
print(os.path.exists('/Users/ellenwu/homework2-2/data/*/input/HCRISv2010/HCRISv2010'))

input("input to continue")
# Call individual scripts -------------------------------------------------
exec(open('/Users/ellenwu/homework2-2/submission3/data-code/H1_HCRISv1996.py').read())
exec(open('/Users/ellenwu/homework2-2/submission3/data-code/H1_HCRISv2010.py').read())

True
True


In [3]:
# Read and combine data ---------------------------------------------------
final_hcris_v1996 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v1996.csv')
final_hcris_v2010 = pd.read_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_v2010.csv')

In [4]:
# Create missing variables for columns introduced in v2010
final_hcris_v1996['hvbp_payment'] = np.nan
final_hcris_v1996['hrrp_payment'] = np.nan

In [5]:
# Combine v1996 and v2010 datasets
final_hcris = pd.concat([final_hcris_v1996, final_hcris_v2010])

In [6]:
# Convert date columns to datetime format
for col in ['fy_end', 'fy_start', 'date_processed', 'date_created']:
    final_hcris[col] = pd.to_datetime(final_hcris[col], format='%m/%d/%Y')

In [7]:
# Convert to absolute values
final_hcris['tot_discounts'] = final_hcris['tot_discounts'].abs()
final_hcris['hrrp_payment'] = final_hcris['hrrp_payment'].abs()

In [8]:
# Extract fiscal year and sort
final_hcris['fyear'] = final_hcris['fy_end'].dt.year
final_hcris = final_hcris.sort_values(by=['provider_number', 'fyear']).drop(columns=['year'], errors='ignore')

In [9]:
# Count hospitals per year
hospital_counts = final_hcris.groupby('fyear').size()

In [10]:
# Clean data --------------------------------------------------------------

# Create count of reports by hospital fiscal year
final_hcris['total_reports'] = final_hcris.groupby(['provider_number', 'fyear'])['provider_number'].transform('count')
final_hcris['report_number'] = final_hcris.groupby(['provider_number', 'fyear']).cumcount() + 1

In [11]:

# Identify hospitals with one report per year
unique_hcris1 = final_hcris[final_hcris['total_reports'] == 1].drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status'], errors='ignore')
unique_hcris1['source'] = 'unique reports'
print(unique_hcris1)

      provider_number   fy_start     fy_end date_processed date_created  \
242             10001 2009-10-01 2010-09-30     2011-08-09   2011-08-05   
24              10005 2009-10-01 2010-09-30     2011-03-09   2011-03-07   
17              10007 2009-10-01 2010-09-30     2011-02-28   2011-02-28   
162             10008 2010-01-01 2010-12-31     2011-06-15   2011-06-14   
2844            10009 2010-02-01 2011-01-31     2014-07-30   2014-07-25   
...               ...        ...        ...            ...          ...   
3118           673035 2010-01-01 2010-12-31     2014-09-23   2014-09-18   
1653           673037 2010-01-01 2010-12-31     2013-06-13   2013-06-13   
369            673038 2009-10-01 2010-09-30     2011-12-19   2011-12-08   
3838           673039 2009-10-30 2010-12-31     2015-06-18   2015-06-17   
373            673040 2010-04-06 2010-12-31     2011-12-19   2011-12-16   

       beds   tot_charges  tot_discounts  tot_operating_exp   ip_charges  ...  \
242   408.0  1.116

In [12]:
# Identify hospitals with multiple reports per year
duplicate_hcris = final_hcris[final_hcris['total_reports'] > 1].copy()
print(duplicate_hcris)

       report  provider_number  npi   fy_start     fy_end date_processed  \
2440   629478            10001  NaN 2016-10-01 2017-09-30     2018-07-19   
8561   629478            10001  NaN 2016-10-01 2017-09-30     2018-07-19   
347    620725            10005  NaN 2016-10-01 2017-09-30     2018-03-19   
6468   620725            10005  NaN 2016-10-01 2017-09-30     2018-03-19   
4767   639416            10006  NaN 2017-07-01 2018-06-30     2018-12-20   
...       ...              ...  ...        ...        ...            ...   
8708   629813           673062  NaN 2017-01-01 2017-12-31     2018-07-19   
5065   640155           673063  NaN 2017-08-01 2018-07-31     2019-01-24   
11186  640155           673063  NaN 2017-08-01 2018-07-31     2019-01-24   
3052   632312           673064  NaN 2017-04-01 2018-03-31     2018-09-17   
9173   632312           673064  NaN 2017-04-01 2018-03-31     2018-09-17   

      date_created  status   beds   tot_charges  ...                   street  \
2440  

In [13]:
# Calculate elapsed time between fy start and fy end for hospitals with multiple reports
duplicate_hcris['time_diff'] = (duplicate_hcris['fy_end'] - duplicate_hcris['fy_start']).dt.days
duplicate_hcris['total_days'] = duplicate_hcris.groupby(['provider_number', 'fyear'])['time_diff'].transform('sum')

In [14]:
# Hospitals where total days < 370, sum values
unique_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] < 370].groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris2['source'] = 'total for year'

In [15]:
# Hospitals with reports exceeding 370 days
duplicate_hcris2 = duplicate_hcris[duplicate_hcris['total_days'] >= 370].copy()
duplicate_hcris2['max_days'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['time_diff'].transform('max')
duplicate_hcris2['max_date'] = duplicate_hcris2.groupby(['provider_number', 'fyear'])['fy_end'].transform('max')

In [16]:
# Primary report selection
unique_hcris3 = duplicate_hcris2[(duplicate_hcris2['max_days'] == duplicate_hcris2['time_diff']) &
                                  (duplicate_hcris2['time_diff'] > 360) &
                                  (duplicate_hcris2['max_date'] == duplicate_hcris2['fy_end'])]
unique_hcris3 = unique_hcris3.drop(columns=['report', 'total_reports', 'report_number', 'npi', 'status', 'max_days', 'time_diff', 'total_days', 'max_date'], errors='ignore')
unique_hcris3['source'] = 'primary report'

In [17]:
# Remaining hospitals with reports covering more than one full year
duplicate_hcris3 = duplicate_hcris2[~duplicate_hcris2.index.isin(unique_hcris3.index)].copy()
duplicate_hcris3['time_diff'] = duplicate_hcris3['time_diff'].astype(int)
duplicate_hcris3['total_days'] = duplicate_hcris3['total_days'].astype(int)

cols_to_weight = ['tot_charges', 'tot_discounts', 'tot_operating_exp', 'ip_charges',
                  'icu_charges', 'ancillary_charges', 'tot_discharges', 'mcare_discharges',
                  'mcaid_discharges', 'tot_mcare_payment', 'secondary_mcare_payment',
                  'hvbp_payment', 'hrrp_payment']

duplicate_hcris3[cols_to_weight] = duplicate_hcris3[cols_to_weight].mul(
    duplicate_hcris3['time_diff'] / duplicate_hcris3['total_days'], axis=0)

In [18]:
# Weighted average
unique_hcris4 = duplicate_hcris3.groupby(['provider_number', 'fyear']).agg({
    'beds': 'max', 'tot_charges': 'sum', 'tot_discounts': 'sum',
    'tot_operating_exp': 'sum', 'ip_charges': 'sum', 'icu_charges': 'sum',
    'ancillary_charges': 'sum', 'tot_discharges': 'sum', 'mcare_discharges': 'sum',
    'mcaid_discharges': 'sum', 'tot_mcare_payment': 'sum',
    'secondary_mcare_payment': 'sum', 'hvbp_payment': 'sum', 'hrrp_payment': 'sum',
    'fy_start': 'min', 'fy_end': 'max', 'date_processed': 'max', 'date_created': 'min',
    'street': 'first', 'city': 'first', 'state': 'first',
    'zip': 'first', 'county': 'first'
}).reset_index()
unique_hcris4['source'] = 'weighted_average'

In [19]:
# Combine final datasets
final_hcris_data = pd.concat([unique_hcris1, unique_hcris2, unique_hcris3, unique_hcris4])
final_hcris_data = final_hcris_data.rename(columns={'fyear': 'year'}).sort_values(by=['provider_number', 'year'])

In [20]:
# Save final data ---------------------------------------------------------
final_hcris_data.to_csv('/Users/ellenwu/homework2-2/data/*/output/HCRIS_Data.csv', index=False)